In [1]:
from sage.all import *
from hashlib import sha256
import random
import math
import secrets

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding

In [2]:
class KEMElGamal:
    
    def __init__(self, sec_param):
        
        self.p = random_prime(2 ** sec_param)
        #R = Integers(p)
        self.F = GF(self.p)
        self.g = self.F.primitive_element()
           
        #g = primitive_root(p)
        
        self.x = randint(1, self.p-2)
        self.h = pow(self.g, self.x, self.p)
        self.priv_key = self.x
        self.pub_key = (self.p, self.g, self.h)
        
    def encapsulate(self, pub_key):
        
        p, g, h = pub_key
        y = random.randint(1, p-1)
        k = pow(h,y,p)
        s = pow(g,y,p)
        return k, s
        
    def decapsulate(self, priv_key, ciphertext):
        s, t = ciphertext
        k = pow(s, priv_key, self.p)
        return k

In [3]:
class PKE_FujisakiOkamoto:

    def __init__(self, sec_param, kem):
        self.kem = kem
        self.key_size = sec_param

    def encrypt(self, pub_key, msg):
        k, s = self.kem.encapsulate(pub_key)
        aes_key = int(k).to_bytes(self.key_size, byteorder='big')
        iv = secrets.token_bytes(self.key_size)
        cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv))
        padder = padding.PKCS7(self.key_size * 8).padder()
        padded_msg = padder.update(msg) + padder.finalize()
        encryptor = cipher.encryptor()
        ct = encryptor.update(padded_msg) + encryptor.finalize()
        c = (s, ct, iv)
        return c

    def decrypt(self, priv_key, ct):
        s, ct, iv = ct
        k = self.kem.decapsulate(priv_key, (s, iv))
        aes_key = int(k).to_bytes(self.key_size, byteorder='big')
        cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv))
        decryptor = cipher.decryptor()
        unpadded_msg = decryptor.update(ct) + decryptor.finalize()
        unpadder = padding.PKCS7(self.key_size * 8).unpadder()
        msg = unpadder.update(unpadded_msg) + unpadder.finalize()
        return msg

In [4]:
kem = KEMElGamal(128)

pke = PKE_FujisakiOkamoto(16, kem)

pub_key = pke.kem.pub_key
priv_key = pke.kem.priv_key

msg = "Estruturas Criptograficas - Grupo 17!"
msg_encoded = msg.encode()

ciphertext = pke.encrypt(pub_key, msg_encoded)

decrypted_msg = pke.decrypt(priv_key, ciphertext)
msg_decoded = decrypted_msg.decode()

print("Mensagem Original: ", msg)
# print("Ciphertext: ", ciphertext)
print("Mensagem Decifrada: ", msg_decoded)
print("A mensagem original e a mensagem decifrada são iguais (True), diferentes (False)")
print(msg == msg_decoded)

Mensagem Original:  Estruturas Criptograficas - Grupo 17!
Mensagem Decifrada:  Estruturas Criptograficas - Grupo 17!
A mensagem original e a mensagem decifrada são iguais (True), diferentes (False)
True
